In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
import scipy.stats as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xg
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from eli5.sklearn import PermutationImportance
from eli5 import show_weights
from sklearn.tree import DecisionTreeRegressor
from bayes_opt import BayesianOptimization

In [112]:
#Загрузим данные из файла.
df = pd.read_excel('/home/rik/Рабочий стол/МИФИ/учеба/мо/curse/data.xlsx', index_col=0)
df.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [113]:
print("train.shape = {} rows, {} cols".format(*df.shape))

train.shape = 1001 rows, 213 cols


# Предобработка данных.

In [114]:
def transform(df): 
    discrete_feature = [feature for feature in df.columns if len(df[feature].unique())<25]
    count_discrete_feature = {}
    for i in tqdm(discrete_feature):
        cnt = 0
        for j in df[i]:
            if j != 0:
                cnt += 1
        count_discrete_feature[f'{i}'] = cnt
    count_discrete_feature = pd.Series(count_discrete_feature)
    df = df.drop(columns=count_discrete_feature[count_discrete_feature == 0].index.to_list())
    df = df.dropna()
    feature_del = ['BertzCT', 'Chi0', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
     'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan3', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Kappa1', 'Kappa2',
     'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MolMR', 'MolWt', 'NHOHCount', 'NumAromaticCarbocycles',
     'NumHAcceptors', 'NumHeteroatoms', 'NumSaturatedCarbocycles', 'NumValenceElectrons', 'SMR_VSA4', 'SMR_VSA9',
     'SlogP_VSA6', 'TPSA', 'VSA_EState2', 'VSA_EState3', 'VSA_EState6', 'fr_Al_OH_noTert', 'fr_COO', 'fr_COO2',
     'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_nitro_arom_nonortho', 'fr_phenol', 'fr_phenol_noOrthoHbond',
    'MinAbsPartialCharge', 'Chi2n']
    df = df.drop(columns=feature_del)   
    return df
    

In [115]:
df = transform(df)

100%|███████████████████████████████████████| 110/110 [00:00<00:00, 9546.71it/s]


## Обучение нескольких моделей, их сравнение. 

In [116]:
target = df['CC50, mM']
data = df.drop(['CC50, mM','IC50, mM','SI'], axis=1)

In [117]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 798 rows, 144 cols
x_valid.shape = 200 rows, 144 cols


In [167]:
# Создадим модель дерева решений
tree = DecisionTreeRegressor(random_state=27)
# Создадим модель CatBoost
model = CatBoostRegressor(random_state=27)
# Создадим модель lightgbm
model_LGBM = LGBMRegressor(random_state=27)
# Создадим модель xgboost
xgb_r = xg.XGBRegressor() 
# обучение моделей
tree.fit(x_train, y_train)
model.fit(x_train, y_train, verbose=0)
model_LGBM.fit(x_train, y_train, verbose=0)
xgb_r.fit(x_train, y_train, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_tree_train = tree.predict(x_train)
y_pred_сat_Boost_train = model.predict(x_train)
y_pred_LGBM_train = model_LGBM.predict(x_train)
y_pred_xgb_train = xgb_r.predict(x_train)
# предсказание ответов для тестовой выборки
y_pred_tree = tree.predict(x_valid)
y_pred_сat_Boost = model.predict(x_valid)
y_pred_LGBM = model_LGBM.predict(x_valid)
y_pred_xgb = xgb_r.predict(x_valid)

print("Train tree R2: ", r2_score(y_train, y_pred_tree_train))
print("Train tree RMSE: ", np.sqrt(mse(y_train, y_pred_tree_train)))
print("Validation tree R2: ", r2_score(y_valid, y_pred_tree))
print("Validation tree RMSE: ", np.sqrt(mse(y_valid, y_pred_tree)))
print('*' * 50)
print("Train сat_Boost R2: ", r2_score(y_train, y_pred_сat_Boost_train))
print("Train сat_Boost RMSE: ", np.sqrt(mse(y_train, y_pred_сat_Boost_train)))
print("Validation сat_Boost R2: ", r2_score(y_valid, y_pred_сat_Boost))
print("Validation сat_Boost RMSE: ", np.sqrt(mse(y_valid, y_pred_сat_Boost)))
print('*' * 50)
print("Train LGBM R2: ", r2_score(y_train, y_pred_LGBM_train))
print("Train LGBM RMSE: ", np.sqrt(mse(y_train, y_pred_LGBM_train)))
print("Validation LGBM R2: ", r2_score(y_valid, y_pred_LGBM))
print("Validation LGBM RMSE: ", np.sqrt(mse(y_valid, y_pred_LGBM)))
print('*' * 50)
print("Train xgb R2: ", r2_score(y_train, y_pred_xgb_train))
print("Train xgb RMSE: ", np.sqrt(mse(y_train, y_pred_xgb_train)))
print("Validation xgb R2: ", r2_score(y_valid, y_pred_xgb))
print("Validation xgb RMSE: ", np.sqrt(mse(y_valid, y_pred_xgb)))

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train tree R2:  0.9055740350082886
Train tree RMSE:  195.29184705080226
Validation tree R2:  0.28799057438692444
Validation tree RMSE:  561.5119175721105
**************************************************
Train сat_Boost R2:  0.8856627872916704
Train сat_Boost RMSE:  214.89791282487985
Validation сat_Boost R2:  0.5905079202955197
Validation сat_Boost RMSE:  425.83255455340566
**************************************************
Train LGBM R2:  0.8742125414785463
Train LGBM RMSE:  225.4016382770237
Validation LGBM R2:  0.5758412333608947
Validation LGBM RMSE:  433.39144129599543
**************************************************
Train xgb R2:  0.9054290360426936
Train xgb RMSE:  195.44173301345927
Validation xgb R2:  0.5517135010469725
Validation xgb RMSE:  445.54742337134417


Из приведенных данных выше мы видем переобучение всех моделей. Наилучшие результаты показали модели СatBoost, LGBM и xgboost. С ними и продолжим работать.

# Подбор признаков, их анализ и оценка важности.

**CatBoost**

In [119]:
col = x_train.columns.tolist()
estimator_cat_boost = PermutationImportance(
    model, scoring= "neg_mean_squared_error", random_state=27
)
importance_cat_boost = estimator_cat_boost.fit(x_train, y_train)



In [120]:
show_weights(importance_cat_boost, feature_names=col, top=x_train.shape[1])

Weight,Feature
35355.1482 ± 3573.7567,Ipc
10385.7895 ± 2497.6304,SPS
10301.6001 ± 1508.4186,BCUT2D_MWLOW
7718.0027 ± 1335.6565,BCUT2D_MRLOW
7290.3922 ± 2392.3029,PEOE_VSA7
7190.1590 ± 1646.5887,Chi0n
7108.4988 ± 1749.1899,FpDensityMorgan2
6805.5754 ± 1486.6095,VSA_EState4
6385.1130 ± 1465.0016,MolLogP
6044.4712 ± 1800.1135,PEOE_VSA6


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [162]:
feature_cat_boost = ['Ipc','SPS','BCUT2D_MWLOW','BCUT2D_MRLOW','PEOE_VSA7','Chi0n','FpDensityMorgan2','VSA_EState4',
'MolLogP','PEOE_VSA6','EState_VSA7','BalabanJ','BCUT2D_MWHI','MinAbsEStateIndex','VSA_EState5','BCUT2D_LOGPLOW',
'qed','BCUT2D_MRHI','BCUT2D_CHGLO','EState_VSA4','AvgIpc','PEOE_VSA8','FractionCSP3','VSA_EState7','EState_VSA3',
'MaxEStateIndex','VSA_EState8','PEOE_VSA10','MinPartialCharge','SMR_VSA7','MaxPartialCharge','EState_VSA2',
'RingCount','EState_VSA9','SlogP_VSA5','SMR_VSA10','HallKierAlpha','VSA_EState10','fr_ketone_Topliss','SlogP_VSA2',
'SlogP_VSA1','SMR_VSA6','MinEStateIndex','PEOE_VSA9','BCUT2D_LOGPHI','EState_VSA8','fr_bicyclic','VSA_EState1',
'fr_quatN','NumRotatableBonds','SMR_VSA5','PEOE_VSA1','fr_urea','fr_Ar_OH','NumHDonors','BCUT2D_CHGHI','EState_VSA6',
'fr_unbrch_alkane','SMR_VSA1','fr_para_hydroxylation','fr_furan','fr_allylic_oxid','SlogP_VSA4','NumAliphaticCarbocycles',
'SMR_VSA3','VSA_EState9','EState_VSA5','fr_Ar_N','SlogP_VSA3','EState_VSA1','SlogP_VSA10','fr_NH2']

In [163]:
target_cb = df['CC50, mM']
data_cb = df[feature_cat_boost]

In [164]:
x_train_cb, x_valid_cb = train_test_split(
    data_cb, train_size=0.8, random_state=1
)
y_train_cb, y_valid_cb = train_test_split(
    target_cb, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_cb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_cb.shape))

x_train.shape = 798 rows, 72 cols
x_valid.shape = 200 rows, 72 cols


In [165]:
model = CatBoostRegressor(random_state=27)
model.fit(x_train_cb, y_train_cb, verbose=100)
y_pred_cb = model.predict(x_train_cb)
y_val_cb = model.predict(x_valid_cb)
print("Train R2: ", r2_score(y_train_cb, y_pred_cb))
print("Train RMSE: ", np.sqrt(mse(y_train_cb, y_pred_cb)))
print("Validation R2: ", r2_score(y_valid_cb, y_val_cb))
print("Validation RMSE: ", np.sqrt(mse(y_valid_cb, y_val_cb)))

Learning rate set to 0.039509
0:	learn: 627.6609601	total: 9.28ms	remaining: 9.27s
100:	learn: 368.3404974	total: 696ms	remaining: 6.19s
200:	learn: 319.3061784	total: 1.57s	remaining: 6.24s
300:	learn: 289.0544665	total: 2.24s	remaining: 5.21s
400:	learn: 263.6227438	total: 2.91s	remaining: 4.35s
500:	learn: 247.7340263	total: 3.63s	remaining: 3.61s
600:	learn: 236.6839763	total: 4.35s	remaining: 2.89s
700:	learn: 227.1471181	total: 5.04s	remaining: 2.15s
800:	learn: 220.5871066	total: 5.73s	remaining: 1.42s
900:	learn: 216.2869873	total: 6.4s	remaining: 703ms
999:	learn: 212.9229874	total: 7.07s	remaining: 0us
Train R2:  0.8877546629717422
Train RMSE:  212.92298755141863
Validation R2:  0.594010992670641
Validation RMSE:  424.00721243788956


Отобрав признаки мы немного улучшили метрики для модели CatBoost на вадидации R2:  0.59, RMSE:  424.

**lightgbm**

In [168]:
col = x_train.columns.tolist()
estimator_LGBM = PermutationImportance(
    model_LGBM, scoring= "neg_mean_squared_error", random_state=27
)
importance_LGBM = estimator_LGBM.fit(x_train, y_train)


In [169]:
show_weights(importance_LGBM, feature_names=col, top=x_train.shape[1])

Weight,Feature
86805.6837 ± 10417.5281,Ipc
28536.0397 ± 4068.0373,BCUT2D_MWHI
27503.2086 ± 4760.2399,Chi0n
27304.1577 ± 4113.7788,FpDensityMorgan2
21595.9432 ± 2126.6813,BCUT2D_MWLOW
19553.2721 ± 4332.6406,VSA_EState4
17840.8281 ± 1896.7853,SPS
15267.2154 ± 1251.5122,BCUT2D_MRLOW
11902.2689 ± 2637.0086,BCUT2D_CHGLO
9829.3876 ± 3739.0949,qed


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [170]:
feature_LGBM = ['Ipc','BCUT2D_MWHI','Chi0n','FpDensityMorgan2','BCUT2D_MWLOW','VSA_EState4','SPS','BCUT2D_MRLOW',
'BCUT2D_CHGLO','qed','VSA_EState7','MolLogP','BCUT2D_CHGHI','BalabanJ','PEOE_VSA7','SlogP_VSA2','PEOE_VSA6',
'AvgIpc','BCUT2D_LOGPLOW','MaxPartialCharge','EState_VSA4','NumHDonors','VSA_EState5','EState_VSA7','BCUT2D_MRHI',
'VSA_EState8','MinAbsEStateIndex','PEOE_VSA8','MinPartialCharge','SlogP_VSA5','EState_VSA5','MaxEStateIndex',
'PEOE_VSA1','PEOE_VSA9','EState_VSA9','SMR_VSA1','HallKierAlpha','EState_VSA3','EState_VSA8','MinEStateIndex',
'SMR_VSA7','PEOE_VSA10','EState_VSA2','BCUT2D_LOGPHI','NumSaturatedHeterocycles','fr_Ar_NH','FractionCSP3',
'SMR_VSA5','VSA_EState1','SMR_VSA10','SMR_VSA6','fr_NH2'
]

In [171]:
target_LGBM = df['CC50, mM']
data_LGBM = df[feature_LGBM]

In [172]:
x_train_LGBM, x_valid_LGBM = train_test_split(
    data_LGBM, train_size=0.8, random_state=1
)
y_train_LGBM, y_valid_LGBM = train_test_split(
    target_LGBM, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_LGBM.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_LGBM.shape))

x_train.shape = 798 rows, 52 cols
x_valid.shape = 200 rows, 52 cols


In [173]:
model_LGBM = LGBMRegressor(random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM, verbose=100)

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMRegressor(random_state=27)

In [174]:
y_pred_LGBM = model_LGBM.predict(x_train_LGBM)
y_val_LGBM = model_LGBM.predict(x_valid_LGBM)
print("Train R2: ", r2_score(y_train_LGBM, y_pred_LGBM))
print("Train RMSE: ", np.sqrt(mse(y_train_LGBM, y_pred_LGBM)))
print("Validation R2: ", r2_score(y_valid_LGBM, y_val_LGBM))
print("Validation RMSE: ", np.sqrt(mse(y_valid_LGBM, y_val_LGBM)))

Train R2:  0.8742942334664948
Train RMSE:  225.32843344903932
Validation R2:  0.5800158032959024
Validation RMSE:  431.25344880621077


Отобрав признаки мы улучшили метрики и немного снизили переобучение для модели lightgbm на вадидации R2:  0.58, RMSE: 431.25.

**xgboost**

In [175]:
col = x_train.columns.tolist()
estimator_xgb = PermutationImportance(
    xgb_r, scoring= "neg_mean_squared_error", random_state=27
)
importance_xgb = estimator_xgb.fit(x_train, y_train)

In [176]:
show_weights(importance_xgb, feature_names=col, top=x_train.shape[1])

Weight,Feature
104423.1854 ± 11165.2703,Ipc
42556.7670 ± 5700.8823,FpDensityMorgan2
28855.0915 ± 4979.4016,BCUT2D_MRLOW
24628.3415 ± 6210.5433,BCUT2D_MWLOW
23458.8289 ± 3598.6015,MaxPartialCharge
19752.5116 ± 2994.7385,PEOE_VSA7
15086.5992 ± 3694.4161,MinEStateIndex
11800.6925 ± 5872.0161,VSA_EState4
9918.9244 ± 1856.8218,BCUT2D_MWHI
9753.8011 ± 2547.3974,qed


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [177]:
feature_xgb = ['MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MaxPartialCharge',
 'MinPartialCharge', 'FpDensityMorgan2', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'BalabanJ', 'Chi0n', 'HallKierAlpha',
 'Ipc', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA2', 'PEOE_VSA3',
 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA3',
 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', 'SlogP_VSA2',
 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA8', 'EState_VSA1', 'EState_VSA10', 'EState_VSA2', 'EState_VSA3',
 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7', 'EState_VSA8', 'EState_VSA9', 'VSA_EState1',
 'VSA_EState10', 'VSA_EState4', 'VSA_EState5', 'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'FractionCSP3',
 'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticRings', 'NumAromaticHeterocycles', 'NumHDonors', 'NumRotatableBonds',
 'NumSaturatedHeterocycles', 'MolLogP', 'fr_Al_OH', 'fr_ArN', 'fr_Ar_N', 'fr_Ar_NH', 'fr_Ar_OH', 'fr_Imine',
 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_allylic_oxid', 'fr_amide', 'fr_aniline', 'fr_aryl_methyl', 'fr_bicyclic',
 'fr_ester', 'fr_furan', 'fr_halogen', 'fr_ketone_Topliss', 'fr_lactone', 'fr_nitrile', 'fr_para_hydroxylation',
 'fr_piperdine', 'fr_piperzine', 'fr_priamide', 'fr_pyridine', 'fr_unbrch_alkane', 'fr_urea']

In [178]:
target_xgb = df['CC50, mM']
data_xgb = data[feature_xgb]

In [179]:
x_train_xgb, x_valid_xgb = train_test_split(
    data_xgb, train_size=0.8, random_state=1
)
y_train_xgb, y_valid_xgb = train_test_split(
    target_xgb, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_xgb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_xgb.shape))

x_train.shape = 798 rows, 101 cols
x_valid.shape = 200 rows, 101 cols


In [180]:
model_xgb = xg.XGBRegressor(random_state=27)
model_xgb.fit(x_train_xgb, y_train_xgb, verbose=100)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=27, ...)

In [181]:
y_pred_xgb = model_xgb.predict(x_train_xgb)
y_val_xgb = model_xgb.predict(x_valid_xgb)
print("Train R2: ", r2_score(y_train_xgb, y_pred_xgb))
print("Train RMSE: ", np.sqrt(mse(y_train_xgb, y_pred_xgb)))
print("Validation R2: ", r2_score(y_valid_xgb, y_val_xgb))
print("Validation RMSE: ", np.sqrt(mse(y_valid_xgb, y_val_xgb)))

Train R2:  0.9054755646197629
Train RMSE:  195.39364878052143
Validation R2:  0.5605202569720857
Validation RMSE:  441.14924236832917


Отобрав признаки мы улучшили метрики для модели xgboost на вадидации R2: 0.56, RMSE: 441.15.

# Подбор гиперпараметров

**CatBoost**

In [200]:
def catboost_cv(depth, learning_rate,min_data_in_leaf):
    depth = int(depth)
    min_data_in_leaf = int(min_data_in_leaf)
    model = CatBoostRegressor(
        depth=depth,
        learning_rate=learning_rate,
        min_data_in_leaf=min_data_in_leaf,
        early_stopping_rounds=10,
        random_seed=27
    )
    model.fit(x_train_cb, y_train_cb, verbose=100)

    return r2_score(y_valid_cb, model.predict(x_valid_cb))

In [202]:
param_space = {
    'depth': (5, 10),             
    'learning_rate': (0.001, 0.5),
    'min_data_in_leaf': (1, 10)  
}

bayesian_opt = BayesianOptimization(
    f=catboost_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=20)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   |   depth   | learni... | min_da... |
-------------------------------------------------------------
0:	learn: 560.8242559	total: 31.3ms	remaining: 31.3s
100:	learn: 204.7246990	total: 1.44s	remaining: 12.9s
200:	learn: 195.9745415	total: 2.85s	remaining: 11.3s
300:	learn: 195.3544493	total: 4.13s	remaining: 9.6s
400:	learn: 195.2987024	total: 5.39s	remaining: 8.05s
500:	learn: 195.2926551	total: 7.21s	remaining: 7.18s
600:	learn: 195.2919362	total: 8.65s	remaining: 5.74s
700:	learn: 195.2918597	total: 10.3s	remaining: 4.37s
800:	learn: 195.2918488	total: 11.8s	remaining: 2.92s
900:	learn: 195.2918474	total: 13.1s	remaining: 1.44s
999:	learn: 195.2918472	total: 14.5s	remaining: 0us
| 1         | 0.5376    | 7.129     | 0.4075    | 7.619     |
0:	learn: 594.7165403	total: 165ms	remaining: 2m 45s
100:	learn: 216.9116513	total: 6.67s	remaining: 59.4s
200:	learn: 198.4434023	total: 12.8s	remaining: 50.8s
300:	learn: 195.8117726	total: 18.6s	remaining: 43.3s
400:	lear

900:	learn: 195.2919327	total: 3.61s	remaining: 397ms
999:	learn: 195.2918645	total: 4.17s	remaining: 0us
| 12        | 0.5577    | 5.319     | 0.467     | 1.006     |
0:	learn: 565.0392989	total: 5.86ms	remaining: 5.85s
100:	learn: 215.7387500	total: 455ms	remaining: 4.05s
200:	learn: 198.9883605	total: 854ms	remaining: 3.39s
300:	learn: 196.0605110	total: 1.23s	remaining: 2.85s
400:	learn: 195.4746753	total: 1.62s	remaining: 2.42s
500:	learn: 195.3355897	total: 2.19s	remaining: 2.18s
600:	learn: 195.3030037	total: 2.6s	remaining: 1.73s
700:	learn: 195.2944053	total: 3.01s	remaining: 1.28s
800:	learn: 195.2925775	total: 3.75s	remaining: 932ms
900:	learn: 195.2920540	total: 4.28s	remaining: 470ms
999:	learn: 195.2919043	total: 4.71s	remaining: 0us
| 13        | 0.5473    | 5.199     | 0.4405    | 3.747     |
0:	learn: 588.9967287	total: 8.8ms	remaining: 8.79s
100:	learn: 236.1246385	total: 1.54s	remaining: 13.7s
200:	learn: 203.7978746	total: 3.14s	remaining: 12.5s
300:	learn: 197.6850

800:	learn: 195.2919163	total: 9.05s	remaining: 2.25s
900:	learn: 195.2918600	total: 9.88s	remaining: 1.09s
999:	learn: 195.2918498	total: 10.9s	remaining: 0us
| 24        | 0.5697    | 6.315     | 0.3854    | 2.036     |
0:	learn: 622.6219113	total: 42.5ms	remaining: 42.4s
100:	learn: 300.2665085	total: 3.46s	remaining: 30.8s
200:	learn: 249.7862984	total: 6.08s	remaining: 24.2s
300:	learn: 223.0186670	total: 8.81s	remaining: 20.5s
400:	learn: 211.5697917	total: 12.1s	remaining: 18s
500:	learn: 205.5738134	total: 15.6s	remaining: 15.5s
600:	learn: 202.2121065	total: 18.7s	remaining: 12.4s
700:	learn: 199.7436096	total: 21.7s	remaining: 9.27s
800:	learn: 197.9415721	total: 24.5s	remaining: 6.1s
900:	learn: 197.0586823	total: 27.9s	remaining: 3.07s
999:	learn: 196.4153834	total: 30.7s	remaining: 0us
| 25        | 0.5543    | 8.977     | 0.06327   | 5.531     |


In [204]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'min_data_in_leaf'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'depth': 6, 'learning_rate': 0.04881680761654606, 'min_data_in_leaf': 2.3690181003128403}
Best R-squared Score: 0.5820


In [221]:
model = CatBoostRegressor(
        depth=6,
        learning_rate=0.04881680761654606,
        l2_leaf_reg=2,
        early_stopping_rounds=5
    )
model.fit(x_train_cb, y_train_cb, verbose=100)
y_pred_cb = model.predict(x_train_cb)
y_val_cb = model.predict(x_valid_cb)
print("Train R2: ", r2_score(y_train_cb, y_pred_cb))
print("Train RMSE: ", np.sqrt(mse(y_train_cb, y_pred_cb)))
print("Validation R2: ", r2_score(y_valid_cb, y_val_cb))
print("Validation RMSE: ", np.sqrt(mse(y_valid_cb, y_val_cb)))

0:	learn: 623.6163031	total: 10.1ms	remaining: 10.1s
100:	learn: 351.2204042	total: 1.62s	remaining: 14.4s
200:	learn: 298.2389329	total: 3.47s	remaining: 13.8s
300:	learn: 264.2438340	total: 5.07s	remaining: 11.8s
400:	learn: 243.2493383	total: 6.72s	remaining: 10s
500:	learn: 229.8222094	total: 8.49s	remaining: 8.45s
600:	learn: 221.0118105	total: 9.44s	remaining: 6.27s
700:	learn: 213.4348119	total: 10.3s	remaining: 4.41s
800:	learn: 208.6709514	total: 11.1s	remaining: 2.76s
900:	learn: 205.3166105	total: 11.8s	remaining: 1.3s
999:	learn: 202.6475435	total: 12.5s	remaining: 0us
Train R2:  0.8983269403777523
Train RMSE:  202.64754345244063
Validation R2:  0.5617625335190899
Validation RMSE:  440.525303275646


После подбора гиперпараметров мы еще улучшили метрики модели CatBoost на вадидации R2: 0.56, RMSE: 440.52.

**lightgbm**

In [196]:
def LGBM_cv(max_depth, learning_rate, min_data_in_leaf):
    max_depth = int(max_depth)
    min_data_in_leaf = int(min_data_in_leaf)
    model = LGBMRegressor(
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_data_in_leaf=min_data_in_leaf,          
        eval_metric  = 'r2',
        early_stopping_rounds=10,
        random_seed=27
    )
    model.fit(x_train_LGBM, y_train_LGBM, eval_set = (x_valid_LGBM, y_valid_LGBM))
    return r2_score(y_valid_LGBM, model.predict(x_valid_LGBM))

In [197]:
param_space = {
    'max_depth': (3, 15),            
    'learning_rate': (0.001, 0.5),  
    'min_data_in_leaf': (1, 10)  
}

bayesian_opt = BayesianOptimization(
    f=LGBM_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=15)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | min_da... |
-------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[1]	valid_0's l2: 379651
[2]	valid_0's l2: 328633
[3]	valid_0's l2: 286929
[4]	valid_0's l2: 252433
[5]	valid_0's l2: 229967
[6]	valid_0's l2: 219302
[7]	valid_0's l2: 211296
[8]	valid_0's l2: 208266
[9]	valid_0's l2: 207839
[10]	valid_0's l2: 203683
[11]	valid_0's l2: 199627
[12]	valid_0's l2: 197236
[13]	valid_0's l2: 194249
[14]	valid_0's l2: 191807
[15]	valid_0's l2: 190957
[16]	valid_0's l2: 194045
[17]	valid_0's l2: 194693
[18]	valid_0's l2: 193524
[19]	valid_0's l2: 193730
[20]	valid_0's l2: 192540
[21]	valid_0's l2: 192261
[22]	valid_0's l2: 193464
[23]	va

[66]	valid_0's l2: 350001
[67]	valid_0's l2: 348841
[68]	valid_0's l2: 347615
[69]	valid_0's l2: 346811
[70]	valid_0's l2: 345604
[71]	valid_0's l2: 344251
[72]	valid_0's l2: 343469
[73]	valid_0's l2: 342224
[74]	valid_0's l2: 341457
[75]	valid_0's l2: 340225
[76]	valid_0's l2: 339472
[77]	valid_0's l2: 338390
[78]	valid_0's l2: 337079
[79]	valid_0's l2: 336336
[80]	valid_0's l2: 335113
[81]	valid_0's l2: 334395
[82]	valid_0's l2: 333153
[83]	valid_0's l2: 332438
[84]	valid_0's l2: 331244
[85]	valid_0's l2: 330546
[86]	valid_0's l2: 329343
[87]	valid_0's l2: 328302
[88]	valid_0's l2: 327632
[89]	valid_0's l2: 326336
[90]	valid_0's l2: 325323
[91]	valid_0's l2: 324697
[92]	valid_0's l2: 324051
[93]	valid_0's l2: 323059
[94]	valid_0's l2: 321814
[95]	valid_0's l2: 320783
[96]	valid_0's l2: 320188
[97]	valid_0's l2: 319583
[98]	valid_0's l2: 318490
[99]	valid_0's l2: 317551
[100]	valid_0's l2: 316836
| 7         | 0.2845    | 0.004411  | 10.18     | 3.63      |
[LightGBM] [Warning] Unknow

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[1]	valid_0's l2: 317914
[2]	valid_0's l2: 247691
[3]	valid_0's l2: 220514
[4]	valid_0's l2: 214217
[5]	valid_0's l2: 210264
[6]	valid_0's l2: 214970
[7]	valid_0's l2: 213595
[8]	valid_0's l2: 211735
[9]	valid_0's l2: 215502
[10]	valid_0's l2: 214641
[11]	valid_0's l2: 217077
[12]	valid_0's l2: 215635
[13]	valid_0's l2: 217136
[14]	valid_0's l2: 215383
[15]	valid_0's l2: 216436
| 12        | 0.5252    | 0.5       | 7.095     | 5.221     |
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[1]	valid_0's l2: 334966
[2]	valid_0's l2: 264675
[3]	valid_0's l2: 239698
[4]	valid_0's l2: 216302
[5]	valid_0's l2: 207920
[6]	valid_0's l2: 202147
[7]	valid_0's l2: 201316
[8]	valid_0's l2: 202361
[9]	valid_0's l2: 202184
[10]	valid_0's l2: 201721
[11]	valid_0's l2: 203952
[12]	valid_0's l2: 201234
[13]	valid_0's l2: 202734
[14]	valid_0's l2: 206617
[15]	valid_0's l2: 205326
[16]	valid_0's l2: 207148
[17]	valid_0's l2: 207074
[18]	valid_0's l2: 206876
[19]	valid_0's l2: 206770
[20]	valid_0's l2: 206625
[21]	valid_0's l2: 206910
[22]	valid_0's l2: 209558
| 18        | 0.5456    | 0.5       | 11.7      | 6.904     |
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warni

In [198]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'min_data_in_leaf'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.5, 'max_depth': 6.25142717805274, 'min_data_in_leaf': 10.0}
Best R-squared Score: 0.6345


In [199]:
params = {
    'max_depth': 6,            
    'learning_rate': 0.5,  
    'min_data_in_leaf': 10
}
model_LGBM = LGBMRegressor(**params, eval_metric  = 'r2', early_stopping_rounds=10, random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM, eval_set = (x_valid_LGBM, y_valid_LGBM))
y_pred_LGBM = model_LGBM.predict(x_train_LGBM)
y_val_LGBM = model_LGBM.predict(x_valid_LGBM)
print("Train R2: ", r2_score(y_train_LGBM, y_pred_LGBM))
print("Train RMSE: ", np.sqrt(mse(y_train_LGBM, y_pred_LGBM)))
print("Validation R2: ", r2_score(y_valid_LGBM, y_val_LGBM))
print("Validation RMSE: ", np.sqrt(mse(y_valid_LGBM, y_val_LGBM)))

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[1]	valid_0's l2: 338805
[2]	valid_0's l2: 232562
[3]	valid_0's l2: 202695
[4]	valid_0's l2: 183975
[5]	valid_0's l2: 175981
[6]	valid_0's l2: 170504
[7]	valid_0's l2: 167464
[8]	valid_0's l2: 162762
[9]	valid_0's l2: 163561
[10]	valid_0's l2: 161861
[11]	valid_0's l2: 163602
[12]	valid_0's l2: 166211
[13]	valid_0's l2: 164770
[14]	valid_0's l2: 164955
[15]	valid_0's l2: 165876
[16]	valid_0's l2: 166791
[17]	valid_0's l2: 167491
[18]	valid_0's l2: 167529
[19]	valid_0's l2: 168451
[20]	valid_0's l2: 168946
Train R2:  0.8301679941038064
Train RMSE:  261.90763961522936
Validation R2:  0.6344819691472423
Validation RMSE:  402.3189687908689


После подбора гиперпараметров мы еще улучшили метрики и снизили переобучение для модели lightgbm на вадидации R2: 0.63, RMSE: 402.32.

**xgboost**

In [211]:
def XGB_cv(max_depth, learning_rate, min_data_in_leaf):
    max_depth = int(max_depth)
    min_data_in_leaf = int(min_data_in_leaf)
    model = xg.XGBRegressor(
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_data_in_leaf=min_data_in_leaf,
        random_seed=27
    )
    model.fit(x_train_xgb, y_train_xgb)
    return r2_score(y_valid_xgb, model.predict(x_valid_xgb))

In [212]:
param_space = {
    'max_depth': (3, 15),            
    'learning_rate': (0.001, 0.5),  
    'min_data_in_leaf': (1, 10)
}

bayesian_opt = BayesianOptimization(
    allow_duplicate_points=True,f=XGB_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=15)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | min_da... |
-------------------------------------------------------------


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [11:34:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_data_in_leaf", "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 1         | 0.4401    | 0.2134    | 12.78     | 7.619     |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [11:34:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_data_in_leaf", "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 2         | 0.4488    | 0.4341    | 7.601     | 9.815     |


...

| 18        | 0.406     | 0.2261    | 12.44     | 5.064     |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [11:35:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_data_in_leaf", "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 19        | 0.5477    | 0.02693   | 4.602     | 3.293     |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [11:35:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_data_in_leaf", "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 20        | 0.5924    | 0.2267    | 4.518     | 9.375     |


In [213]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'min_data_in_leaf'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.22673431877608324, 'max_depth': 4.518404041896739, 'min_data_in_leaf': 9.375216298657694}
Best R-squared Score: 0.5924


In [214]:
params = {
    'max_depth': 4,            
    'learning_rate': 0.22673431877608324,  
    'min_data_in_leaf': 9       
}
model_XGB = xg.XGBRegressor(**params, random_state=27)
model_XGB.fit(x_train_xgb, y_train_xgb)
y_pred_xgb = model_XGB.predict(x_train_xgb)
y_val_xgb = model_XGB.predict(x_valid_xgb)
print("Train R2: ", r2_score(y_train_xgb, y_pred_xgb))
print("Train RMSE: ", np.sqrt(mse(y_train_xgb, y_pred_xgb)))
print("Validation R2: ", r2_score(y_valid_xgb, y_val_xgb))
print("Validation RMSE: ", np.sqrt(mse(y_valid_xgb, y_val_xgb)))

/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [11:35:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "min_data_in_leaf" } are not used.

  warnings.warn(smsg, UserWarning)


Train R2:  0.8910168555255014
Train RMSE:  209.80607744519384
Validation R2:  0.5923720383391198
Validation RMSE:  424.8621968221902


После подбора гиперпараметров мы еще улучшили метрики и снизили переобучение для модели XGB на вадидации R2: 0.59, RMSE: 424.86.

**Выводы**

Мы сравнили 4 регрессионые модели: DecisionTreeRegressor, CatBoostRegressor, LGBMRegressor, XGBRegressor. Наилучший результат и наименьшее переобучение показала модель lightgbm с результатом на трейне: r2=0.83 и RMSE=261.9 и тесте: r2=0.63 и RMSE=402.32. Данная модель объясняет 63% дисперсии между тренеровочными и тестовыми данными и корень среднеквадратичной ошибки составляет 402.32 микромоль (мкМ). Также хорошую метрику показала модель CatBoostRegressor, но у нее больше переобучение.

**Рекомендации**

Для улучшения значения метрики и снижения переобучения нужно будет увеличить количество данных. Также можно попробовать логафрифмировать целевую переменную.